In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from tqdm import tqdm

import pyaudio  
import wave  

import shutil

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from collections import Counter

# Чтение данных

In [2]:
path = 'data'

In [3]:
df = pd.DataFrame(columns=['name', 'text', 'path'])

In [4]:
count = 0

In [5]:
for i in os.walk(path):
    count+= 1
count

0

In [6]:
# count = 0
a = -1
# for i in tqdm(range(count)):
for root, dirs, files in os.walk(path):
    if len(files) > 1:
        for file in files:
            path_file = root + '\\' + file
            path_name = root + '\\'
#                 print(root)
            if file.split('.')[-1] == 'txt':
                with open(path_file, encoding = 'utf-8') as f:
    #                    print(f.read())
                    dict_ = {'name': file,
                             'text' : f.read(),
                             'path': path_name}
                    df= df.append(dict_, ignore_index=True)
                    if a != i:
                        a = i
#                         print(a)
#                         a = i
#                         print(i)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93244 entries, 0 to 93243
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    93244 non-null  object
 1   text    93244 non-null  object
 2   path    93244 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [8]:
# Удаление дубликатов
df = df.drop_duplicates(subset=['name', 'text', 'path'], keep='last')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93244 entries, 0 to 93243
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    93244 non-null  object
 1   text    93244 non-null  object
 2   path    93244 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


In [9]:
# ф-я приводит слово к нормальной форме
def norm_text(text):
    res = []
    for i in text.strip().split():
        i = morph.parse(i)[0]
        i = i.normal_form
        res.append(i)
    return res

In [10]:
df['text_norm'] = df['text'].apply(lambda x: norm_text(x))

In [11]:
df['join_text_norm'] = df['text_norm'].apply(lambda x: ' '.join(x))

In [12]:
df['full_path'] = [i + j for i, j in list(zip(df['path'], df['name']))]

In [45]:
df['name'] = [str(i).replace('.txt', '') for i in df['name']]

In [46]:
df.sample(3)

,name,text,path,text_norm,join_text_norm,full_path
49847,3ca6115b-c7cc-4255-83da-892a65a8245b,а корпус два квартира двадцать семь этаж четвё...,data\8\9\,"[а, корпус, два, квартира, двадцать, семь, эта...",а корпус два квартира двадцать семь этаж четвё...,data\8\9\3ca6115b-c7cc-4255-83da-892a65a8245b.txt
56735,2312600b-fa7e-40a9-8ef8-83dfc44ca0de,фиг тебе придётся тебя аудио слушать,data\9\c\,"[фига, ты, прийтись, ты, аудио, слушать]",фига ты прийтись ты аудио слушать,data\9\c\2312600b-fa7e-40a9-8ef8-83dfc44ca0de.txt
8385,fbacb529-6985-44c8-a993-b02287592f9e,моя интуиция прислушиваться к своим ощущениям,data\1\6\,"[мой, интуиция, прислушиваться, к, свой, ощуще...",мой интуиция прислушиваться к свой ощущение,data\1\6\fbacb529-6985-44c8-a993-b02287592f9e.txt


In [47]:
df.to_pickle('voice.pkl')

In [7]:
df = pd.read_pickle('voice.pkl')

In [8]:
print(df.shape)
df.sample()

(93244, 6)


,name,text,path,text_norm,join_text_norm,full_path
25097,401e3254-4bc5-4cdf-a889-47cf30dc4ae2,аа на строительстве реальность дохера бабок,data\4\5\,"[аа, на, строительство, реальность, дохер, бабка]",аа на строительство реальность дохер бабка,data\4\5\401e3254-4bc5-4cdf-a889-47cf30dc4ae2.txt


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93244 entries, 0 to 93243
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            93244 non-null  object
 1   text            93244 non-null  object
 2   path            93244 non-null  object
 3   text_norm       93244 non-null  object
 4   join_text_norm  93244 non-null  object
 5   full_path       93244 non-null  object
dtypes: object(6)
memory usage: 5.0+ MB


# Воспроизведение из df

In [10]:
def play_wav(text, path):
    print(text)
    #define stream chunk   
    chunk = 1024  

    #open a wav format music  
    f = wave.open(fr"{path}","rb")  
    #instantiate PyAudio  
    p = pyaudio.PyAudio()  
    #open stream  
    stream = p.open(format = p.get_format_from_width(f.getsampwidth()),  
                    channels = f.getnchannels(),  
                    rate = f.getframerate(),  
                    output = True)  
    #read data  
    data = f.readframes(chunk)  

    #play stream  
    while data:  
        stream.write(data)  
        data = f.readframes(chunk)  

    #stop stream  
    stream.stop_stream()  
    stream.close()  

    #close PyAudio  
    p.terminate() 

In [9]:
find = 'down'
# path = df['path'][df['text'] == find]
name = df['name'][df['text'] == find]
# text = df['text'][df['text'] == find]
# path_name = [i + j.replace('txt', 'wav') for i, j in list(zip(path, name))]
len(name)

0

In [54]:
# if len(path_name) >= 1:
#     for i, j in list(zip(text, path_name)):
#         play_wav(i, j)

точка
точка
точка
точка
точка
точка


# Ф-я преобразования цифр в строчные значения

In [2]:
def as_words(n):
    """Convert an integer n (+ve or -ve) to English words."""
    # lookups
    ones = ['ноль', 'один', 'два', 'три', 'четыре',
            'пять', 'шесть', 'семь', 'восемь', 'девять', 
            'десять', 'одиннадцать', 'двенадцать', 'тринадцать', 'четырнадцать',
            'пятнадцать', 'шестнадцать', 'семнадцать', 'восемнадцать', 'девятнадцать']
    tens = ['ноль', 'десять', 'двадцать', 'тридцать', 'сорок',
            'пятьдесят', 'шестьдесяст', 'семьдесят', 'восемьдесят', 'девяносто']
    # negative case
    if n < 0:
        return "минус {0}".format(as_words(abs(n)))
    # 1000+
    for order, word in [(10**12, "трилион"), (10**9, "биллион"),
                        (10**6, "миллион"), (10**3, "тысячи")]:
        if n >= order:
            return "{0} {1}{2}".format(as_words(n // order), word,
                                       " {0}".format(as_words(n % order))
                                       if n % order else "")
    # 100-999
    if n >= 100:
        if n / 300 > 1:
            return "триста {1}".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 300 == 1:
            return "триста"
        elif n / 200 >= 1:
            return "двести".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 200 == 1:
            return "двести"
        elif n / 100 > 1:
            return "сто {1}".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 100 == 1:
            return "сто"
        
        else:
            return "{0} сто".format(as_words(n // 100))
    # 0-99
    if n < 20:
        return ones[n]
    else:
        return "{0}{1}".format(tens[n // 10],
                               " {0}".format(as_words(n % 10)) 
                               if n % 10 else "") 

In [3]:
main_word = []

In [4]:
word = ['килина', 'ванна','кухня','туалет','коридор','детская','спальня', 'включить', 'выключить']
# word = ['включить', 'выключить', 'точка', 'привет', "мотодепо", 'пока', 'вверх', 'вниз', 'влево', "вправо"]

In [5]:
number = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40,50,60,70,80,90,
         100,200, 300]

In [6]:
len(number)

31

In [7]:
for i in number:
    num = as_words(i)
#     print(num, i)
#     print('*' * 20)
    main_word.append(num)
len(main_word)

31

In [8]:
for i in word:
    main_word.append(i)
len(main_word)    

40

# normal form

## save result_all_data

## эта часть ищет слова из main_word, проверяя каждое слово в тексте к аудио 

In [18]:
df.columns

Index(['name', 'text', 'path', 'text_norm', 'join_text_norm', 'full_path'], dtype='object')

In [18]:
%%time

# Ищу по всем папкам дата сета, нужные файлы копирую в отдельную папку 

dict_word = {}
a = ''
final_folder = 'result_all_data'
for word in tqdm(main_word):
    count = 0
    for j in df.iterrows():
        path = j[1][2]
        name = j[1][0]
        text = j[1][1]
        text_norm = j[1][3]
#         print(path)
        word_norm = morph.parse(word)[0]
        word_norm = word_norm.normal_form
        if word_norm in text_norm:
            count += 1
# #             text = ' '.join(text)
            name_wav = name + '.wav'
            name_txt = name + '.txt'
            path_wav = path + name_wav
            path_txt = path + name_txt

            shutil.copy(path_wav, final_folder)
            shutil.copy(path_txt, final_folder)


    print(word, count)
            

count

  2%|██                                                                                 | 1/41 [00:15<10:30, 15.75s/it]

ноль 159


  5%|████                                                                               | 2/41 [00:54<18:52, 29.03s/it]

один 2079


  7%|██████                                                                             | 3/41 [01:28<19:52, 31.38s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████  | 40/41 [12:33<00:16, 16.62s/it]

влево 7


100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [12:42<00:00, 18.59s/it]

вправо 4
Wall time: 12min 42s


4

In [13]:
df_res = pd.DataFrame(columns=['name', 'text', 'path'])

In [20]:
path = 'result_all_data'
a = -1
# for i in tqdm(range(count)):
for root, dirs, files in os.walk(path):
    if len(files) > 1:
        for file in files:
            path_file = root + '\\' + file
            path_name = root + '\\'
#                 print(root)
            if file.split('.')[-1] == 'txt':
                with open(path_file, encoding = 'utf-8') as f:
    #                    print(f.read())
                    dict_ = {'name': file,
                             'text' : f.read(),
                             'path': path_name}
                    df_res= df_res.append(dict_, ignore_index=True)
                    if a != i:
                        a = i
                        print(a)
#                         a = i
#                         print(i)

вправо


In [22]:
df_res.to_pickle('df_all_res.pkl')

# mini result DF

## эта часть ищет слова из main_word, проверяя только если в аудио произносится одно слово

In [19]:
df.sample()

,name,text,path,text_norm,join_text_norm,full_path
59366,6ec58289-8f16-4581-9e2e-122ae42a16c8,аа здесь определённый участок полимерной основы,data\a\3\,"[аа, здесь, определённый, участок, полимерный,...",аа здесь определённый участок полимерный основа,data\a\3\6ec58289-8f16-4581-9e2e-122ae42a16c8.txt


In [5]:
# Ищу по всем папкам дата сета, нужные файлы копирую в отдельную папку 
count = 0
cnt = 1
final_folder = 'result_data'
for i in main_word:
#     num = as_words(i)
    i_n = morph.parse(i)[0]
    i_n = i_n.normal_form
    list_ = df['join_text_norm'][df['join_text_norm'] == i_n]
    if len(list_) >= 1:
        count += len(list_)
        path = df['path'][df['join_text_norm'] == i_n]
        name = df['name'][df['join_text_norm'] == i_n]
#         name = df['name'][df['text'] == i]
        name_wav = [i + '.wav' for i in name]
        name_txt = [i + '.txt' for i in name]
        path_wav = [i + j for i, j in list(zip(path, name_wav))]
        path_txt = [i + j for i, j in list(zip(path, name_txt))]
        for i, j in list(zip(path_wav, path_txt)):
            shutil.copy(i, final_folder)
            shutil.copy(j, final_folder)
#             print(cnt)
            cnt += 1
        print(f'{i} : {len(list_)}')
count

0

In [9]:
df_res = pd.DataFrame(columns=['name', 'text', 'path'])

In [10]:
path = 'result_data'
count = 0
for root, dirs, files in os.walk(path):
    for i in files:
        count += 1
count

144264

In [13]:
os.path.exists(r"result_data\2\0011d821437ae7e0afae1ac7cc260adb.txt")
file = r"result_data\2\0011d821437ae7e0afae1ac7cc260adb.txt"
with open(file) as f:
    print(f.read())

выключить


In [14]:
%%time

path = 'result_data'
a = -1
# for i in tqdm(range(count)):
for root, dirs, files in os.walk(path):
    if len(files) > 1:
        for file in files:
            path_file = root + '\\' + file
            path_name = root + '\\'
#                 print(root)
            if file.split('.')[-1] == 'txt':
#                 print(path_file)
                try:
                    with open(path_file, encoding = 'utf-8') as f:
        #                    print(f.read())

                        dict_ = {'name': file,
                                 'text' : f.read(),
                                 'path': path_name}
                        df_res= df_res.append(dict_, ignore_index=True)
                        if a != i:
                            a = i
                            print(a)
                except:
                    with open(path_file) as f:
        #                    print(f.read())

                        dict_ = {'name': file,
                                 'text' : f.read(),
                                 'path': path_name}
                        df_res= df_res.append(dict_, ignore_index=True)
                        if a != i:
                            a = i
                            print(a)
#                         a = i
#                         print(i)

kilina_9_tone_0.59.wav


In [28]:
df_res[df_res['text'] == 'килина']

,name,text,path
270,kilina_1.txt,килина,result_data\1\
271,kilina_10.txt,килина,result_data\1\
272,kilina_11.txt,килина,result_data\1\
273,kilina_12.txt,килина,result_data\1\
274,kilina_13.txt,килина,result_data\1\
275,kilina_14.txt,килина,result_data\1\
276,kilina_15.txt,килина,result_data\1\
277,kilina_2.txt,килина,result_data\1\
278,kilina_3.txt,килина,result_data\1\
279,kilina_4.txt,килина,result_data\1\


In [15]:
df_res['name'] = [i.replace('.txt', '') for i in df_res.name]

In [16]:
def norm_text_res(text):
    i_n = morph.parse(text)[0]
    i_n = i_n.normal_form
    return i_n

In [17]:
%%time
df_res['text_norm'] = df_res['text'].apply(lambda x: norm_text_res(x))

Wall time: 13.4 s


In [18]:
df_res.head()

,name,text,path,text_norm
0,00263e5e-6685-4ad6-9b44-c91af6bd73dd,привет,result_data\1\,привет
1,0077f49e-8768-4be1-b8d2-f82a5db33fc8,семьдесят шесть,result_data\1\,семьдесят шесть
2,013b89f4-648e-4bf2-aa75-a5f0445dda64,привет,result_data\1\,привет
3,0199d353-2875-4b53-ae42-58096a57d241,привет,result_data\1\,привет
4,02eec118-37ab-44a7-a09e-860814d79b89,привет,result_data\1\,привет


In [20]:
df_res.shape

(72131, 4)

In [35]:
file_path = r"result_data\1\0199d353-2875-4b53-ae42-58096a57d241.txt"
os.path.exists(file_path)

True

In [21]:
df_res.to_pickle('df_res.pkl')

In [37]:
df_res = pd.read_pickle('df_res.pkl')

In [38]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361 entries, 0 to 1360
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       1361 non-null   object
 1   text       1361 non-null   object
 2   path       1361 non-null   object
 3   text_norm  1361 non-null   object
dtypes: object(4)
memory usage: 42.7+ KB


# проверка конечной папки

In [28]:
path = 'result_data'

In [29]:
df_chek = pd.DataFrame(columns=['name', 'text', 'path'])

In [30]:
# count = 0
a = -1
# for i in tqdm(range(count)):
for root, dirs, files in os.walk(path):
    if len(files) > 1:
        for file in files:
            path_file = root + '\\' + file
            path_name = root + '\\'
#                 print(root)
            if file.split('.')[-1] == 'txt':
                with open(path_file, encoding = 'utf-8') as f:
    #                    print(f.read())
                    dict_ = {'name': file,
                             'text' : f.read(),
                             'path': path_name}
                    df_chek= df_chek.append(dict_, ignore_index=True)
                    if a != i:
                        a = i
                        print(a)
#                         a = i
#                         print(i)

спальня


In [31]:
a = df_chek['text']
a = dict(Counter(a))
for i, j in a.items():
    print(i, j)

привет 158
семьдесят шесть 2
один 32
сорок один 2
два 26
две 2
сорок два 1
ноль 11
двух 1
одиннадцать 1
одна 1
сто сорок два 2
восемьдесят 1
четыре 5
двести семьдесят один 2
тридцать два 1
три 5
сорок шесть 1
сто тридцать восемь 1
тридцать девять 1
двести двадцать 1
сто тридцать девять 1
точка 2
тридцать 1
тридцать шесть 1
пять 1
сто десять 1
тридцать один 1
девятнадцать 1
пока 2
двадцать 1
сто тридцать один 1
килина 15


# Kilina

In [52]:
from pydub import AudioSegment

c:\program files\python37\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [39]:
path = 'килина'
for root, dirs, files in os.walk(path):
    for i in files:
        
        new_name = i.replace('(', '').replace(')', '').replace(' ', '_')
        path_file = path + f"\\{i}"
        os.rename(path + f"\\{i}", path + f"\\{new_name}" )
#         print(new_name)

In [54]:
DATASET_PATH = 'result_data\\'

In [64]:
for root, dirs, files in os.walk(DATASET_PATH):
    for i in files:
#         print(i.split('.')[0][:6])
        if i.split('.')[1] == 'wav' and i.split('.')[0][:6] == 'kilina':
            sound = AudioSegment.from_wav(DATASET_PATH + i)
            sound = sound.set_channels(1)
            sound.export(DATASET_PATH + i, format="wav")
            print(i)
        

kilina_1.wav
kilina_10.wav
kilina_11.wav
kilina_12.wav
kilina_13.wav
kilina_14.wav
kilina_15.wav
kilina_2.wav
kilina_3.wav
kilina_4.wav
kilina_5.wav
kilina_6.wav
kilina_7.wav
kilina_8.wav
kilina_9.wav


In [ ]:

sound = AudioSegment.from_wav(DATASET_PATH + 'kilina_1.wav')
sound = sound.set_channels(1)
sound.export(DATASET_PATH + 'kilina_1.wav', format="wav")